In [ ]:
import json
import re

In [ ]:
##############################################
# 1) SARCASM DETECTION
##############################################
# Use a pre-trained Hugging Face model specialized in detecting irony/sarcasm.
#   Model card: https://huggingface.co/cardiffnlp/twitter-roberta-base-irony
# Install: pip install transformers torch
# This model is trained on Twitter data

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

SARCASTIC_LABEL = "sarcastic"
NOT_SARCASTIC_LABEL = "not_sarcastic"

model_name = "cardiffnlp/twitter-roberta-base-irony"
sarcasm_tokenizer = AutoTokenizer.from_pretrained(model_name)
sarcasm_model = AutoModelForSequenceClassification.from_pretrained(model_name)

def detect_sarcasm(text: str) -> str:
    inputs = sarcasm_tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = sarcasm_model(**inputs).logits
    probs = torch.softmax(logits, dim=1).squeeze()
    # Label 0 = non-irony, Label 1 = irony
    if probs[1].item() > 0.5:
        return SARCASTIC_LABEL
    else:
        return NOT_SARCASTIC_LABEL


In [ ]:
# !python -m spacy download en_core_web_trf

In [ ]:
##############################################
# 2) NAMED ENTITY RECOGNITION (NER)
##############################################
# We'll use spaCy's English model
# Install: pip install spacy
# Then: python -m spacy download en_core_web_sm
# Then: python -m spacy download en_core_web_trf

import spacy

# 1) Load an advanced spaCy model (fallback to en_core_web_sm if needed)
try:
    nlp = spacy.load("en_core_web_trf")
    print("Using en_core_web_trf for advanced NER.")
except:

    # dont fall aback to smallermodel for better results
    # nlp = spacy.load("en_core_web_sm")
    print("Falling back to en_core_web_sm.")


def do_ner(text: str):
    """
    Return a list of recognized named entities (text + label).
    Example output: [{"entity": "AI", "label": "ORG"}, ...]
    """
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append({"entity": ent.text, "label": ent.label_})

    return entities

Using en_core_web_trf for advanced NER.


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [ ]:
##############################################
# 3) CONCEPT EXTRACTION
##############################################
# Concept extraction can be domain-specific. 
# that looks for keywords/phrases in the text and groups them under certain "concepts".
# In real practice, you might use a more sophisticated approach (e.g., ontology mapping).


concept_dictionary = {
    "jobs_and_careers": [
        "job", "jobs", "career", "hire", "hiring", "position", "occupation", "employment",
        "recruitment", "headhunting", "resumé", "job search", "job application", "interview",
        "promotion", "layoff", "downsizing", "resignation", "fired", "retrenched", "workforce",
        "unemployment", "job board", "career change", "gig economy", "contract work",
        "freelance", "upskilling", "reskilling", "talent shortage", "skills gap",
        "career transition", "remote job", "flexible work", "hybrid work"
    ],

    "ai_tech": [
        "ai", "artificial intelligence", "ml", "machine learning", "deep learning",
        "neural network", "natural language processing", "NLP", "computer vision",
        "generative ai", "chatgpt", "gpt-4", "llm", "transformer model", "reinforcement learning",
        "automation", "autonomous system", "robot", "ai-powered", "ai-driven",
        "ai algorithm", "ai model", "ai tool", "ai system", "ai platform", "ai assistant",
        "predictive modeling", "intelligent agent", "intelligent automation", "digital worker",
        "AI ethics", "AI governance", "AI safety", "AI alignment", "AI regulation",
        "OpenAI", "Hugging Face", "LangChain", "Chatbot", "BERT", "GPT", "GAN",
        "AI job disruption", "AI replacing jobs", "AI impact", "AI revolution"
    ],

    "job_market_trends": [
        "job market", "labor market", "workforce trends", "employment trends",
        "market shift", "skills demand", "future of work", "career outlook",
        "talent acquisition", "workforce transformation", "job automation",
        "digital transformation", "economic uncertainty", "AI economy", "skills of the future",
        "AI in hiring", "job displacement", "employment shift", "workplace change"
    ],

    "finance": [
        "bank", "investment", "finance", "trading", "fintech", "income", "earnings",
        "financial security", "salary", "paycheck", "minimum wage", "wealth gap",
        "cost of living", "job loss compensation", "recession", "economic downturn",
        "furlough", "financial hardship", "severance", "retirement fund"
    ],

    "mental_health": [
        "stress", "burnout", "anxiety", "job stress", "mental health", "wellbeing",
        "work-life balance", "job insecurity", "career anxiety", "AI anxiety"
    ],

    "education_training": [
        "upskilling", "reskilling", "online course", "certification", "MOOC", "bootcamp",
        "lifelong learning", "career coaching", "learning path", "digital skills",
        "AI literacy", "tech training", "coursework", "workforce development"
    ],

     "automation_and_displacement": [
        "automation", "job automation", "automated process", "workflow automation",
        "displaced workers", "displacement", "automated job loss", "automated replacement",
        "robotic process automation", "RPA", "bots replacing humans", "AI takeover",
        "human redundancy", "task automation"
    ],

    "policy_and_governance": [
        "universal basic income", "UBI", "AI regulation", "labor law", "AI tax",
        "tech policy", "future legislation", "AI governance", "worker protection",
        "job guarantee", "government retraining", "economic policy", "union response",
        "digital rights", "job security policy"
    ],

    "recruitment_technology": [
        "AI recruitment", "AI hiring", "talent filter", "algorithmic hiring",
        "resume screening", "candidate ranking", "job matching platform", "talent intelligence",
        "hiring automation", "virtual interview", "digital HR", "ATS", "HR tech"
    ],

    "remote_and_gig_work": [
        "gig economy", "freelancing", "remote job", "remote-first", "digital nomad",
        "platform work", "side hustle", "Uberization", "creator economy", "independent worker",
        "contractor", "online work", "flex work", "microtask", "Upwork", "Fiverr"
    ],

    "public_sentiment_discourse": [
        "boomer", "doomer", "tech bro", "jobpocalypse", "decel", "quiet quitting",
        "great resignation", "job hopping", "layoff wave", "hustle culture", "AI hype",
        "future fear", "doomscrolling", "LinkedIn post", "upskilling frenzy", "prompt engineering"
    ],

    "tech_company_trends": [
        "OpenAI", "Google DeepMind", "Meta AI", "Anthropic", "Stability AI", "Amazon layoffs",
        "tech layoffs", "hiring freeze", "big tech", "FAANG", "tech exodus", "startup layoffs",
        "VC funding freeze", "AI startup", "early retirement", "LinkedIn hiring trends"
    ]
}






In [ ]:
# ! pip install rapidfuzz


In [ ]:
iimport re
from spacy.matcher import PhraseMatcher
# For fuzzy matching (preferred over fuzzywuzzy)
# pip install rapidfuzz
from rapidfuzz import fuzz
from rapidfuzz import process as fuzz_process

# Preprocess PhraseMatcher for multi-word terms
phrase_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
for cat, keywords in concept_dictionary.items():
    patterns = [nlp.make_doc(kw) for kw in keywords if len(kw.split()) > 1]
    if patterns:
        phrase_matcher.add(cat, patterns)

def lemmatized_tokens(doc):
    return [token.lemma_.lower() for token in doc if token.is_alpha]

def do_concept_extraction_v3(text: str, fuzzy_threshold=88):
    doc = nlp(text)
    matched_concepts = set()

    # Phrase matching (multi-word only)
    matches = phrase_matcher(doc)
    for match_id, start, end in matches:
        matched_concepts.add(nlp.vocab.strings[match_id])

    # Token lemmatization
    tokens = lemmatized_tokens(doc)

    # Fuzzy matching for single-word concepts only
    for cat, keywords in concept_dictionary.items():
        for kw in keywords:
            if len(kw.split()) == 1:
                for token in tokens:
                    score = fuzz.partial_ratio(token, kw.lower())
                    if score >= fuzzy_threshold:
                        matched_concepts.add(cat)
                        break  # Avoid multiple adds per category

    return list(matched_concepts)

In [ ]:
##############################################
# 4) ASPECT EXTRACTION
##############################################
# Aspect extraction typically appears in "Aspect-Based Sentiment Analysis (ABSA)."
# The code below is a *placeholder* showing how you might:
#   1. Identify candidate aspect terms (often nouns or noun phrases)
#   2. Return them as “aspects” for each post
# For a more advanced approach, check out open-source ABSA libraries or spacy patterns.

def do_aspect_extraction(text: str):
    """
    Simple approach: collect noun chunks as 'aspects'.
    Example: "AI is transforming the job market." -> ["AI", "the job market"]
    """
    doc = nlp(text)
    aspects = []
    for chunk in doc.noun_chunks:
        aspects.append(chunk.text.strip())
    return aspects

In [ ]:
'''
Richer Aspect Extraction using/;
Instead of returning every noun_chunk, we can use dependency patterns to identify
aspects that have “descriptive context” (e.g., adjectives or prepositional modifiers).

We can attempt to collect the head noun plus its modifiers (adjectival, compound)
to form a more coherent aspect phrase.
'''
def do_aspect_extraction_v2(text: str):
    """
    Extract aspects by looking for 'noun phrases' that include
    any of the following:
      - Adjectival modifiers (amod)
      - Compound nouns
      - Prepositional modifiers (nmod) linking nouns
    This approach can yield more descriptive aspects:
      e.g., "powerful AI model", "advanced deep learning techniques"
    """
    doc = nlp(text)
    aspects = []

    # We'll store aspect phrases by their head noun token
    visited_heads = set()

    for token in doc:
        # If it's a noun that isn't a pronoun or 'dummy' token
        if token.pos_ == "NOUN" and token.lemma_ not in ["thing", "something", "someone"]:
            if token.dep_ in ["nsubj", "dobj", "pobj", "ROOT", "attr", "conj"]:
                aspect_span = _expand_aspect(token)
                # De-duplicate
                if aspect_span not in visited_heads:
                    visited_heads.add(aspect_span)
                    aspects.append(aspect_span)

    return aspects

def _expand_aspect(head_token):
    """
    Expand a noun token to include its adjectival or compound modifiers.
    We'll gather left-side adjectives, compound nouns, etc.
    Example:
      Head: "model"
      Left modifies: "powerful", "AI"
      => "powerful AI model"
    """
    # Gather all tokens that are part of this aspect
    min_i = head_token.i
    max_i = head_token.i
    for child in head_token.children:
        # If there's a prepositional structure "model of something", you can add logic here
        pass

    # Check left siblings for compounds or amod
    for left in reversed(list(head_token.lefts)):
        if left.dep_ in ["amod", "compound", "nmod", "nn"] or left.pos_ in ["ADJ", "NOUN"]:
            min_i = min(min_i, left.i)
        else:
            # If we hit a token that isn't describing this noun, break out
            break

    # Build the text from min_i to max_i
    aspect_span = head_token.doc[min_i : max_i + 1].text
    return aspect_span

In [ ]:
##############################################
# PROCESS JSON DATA
##############################################
if __name__ == "__main__":
    # REPLACE WITH YOUR ACTUAL FILENAME
    filename = r"/content/drive/MyDrive/x_data/testing_x_data_file.json"
    output_filename = r"/content/drive/MyDrive/x_data/testing_x_output.json"

    # Load JSON
    with open(filename, "r", encoding="utf-8") as infile:
        data = json.load(infile)

    # Process each record
    for i, record in enumerate(data):
        if i % 100 == 0:
            print(f"Processing record {i}/{len(data)}")

        text = record.get("text", "")

        # 1) Sarcasm detection
        record["sarcasm_label"] = detect_sarcasm(text)  # "sarcastic" or "not_sarcastic"

        # 2) Named Entity Recognition
        record["named_entities"] = do_ner(text)

        # 3) Concept Extraction
        record["concepts_v3"] = do_concept_extraction_v3(text)

        # 4) Aspect Extraction
        record["aspects"] = do_aspect_extraction(text)
        record["aspects_v2"] = do_aspect_extraction_v2(text)


    # Save updated JSON
    with open(output_filename, "w", encoding="utf-8") as outfile:
        json.dump(data, outfile, indent=4, ensure_ascii=False)

    print("All done! Check your output file for updated JSON data.")
    print(f"Output saved to {output_filename}")
    print(f"Original data count: {len(data)}")




Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Processing record 0/4
All done! Check your output file for updated JSON data.
Output saved to /content/drive/MyDrive/x_data/testing_x_output.json
Original data count: 4


In [ ]:
import pprint
  # Process and print results
for i, record in enumerate(data[:10]):  # You can remove [:10] to print all
    print("=" * 100)
    print(f"Record #{i+1}")
    print("-" * 100)
    print(f"📌 Text:\n")
    pprint.pprint(f"{record['text']}")

    print("="*10)
    print(f"🌀 Sarcasm Detection: {record.get('sarcasm_label', 'N/A')}")

    print("="*10)
    print("\n🔍 Named   Entity Recignition(spaCy):")
    for ent in record.get("named_entities", []):
        print(f"  - {ent['entity']} ({ent['label']})")

    print("="*10)
    print("\n🧠 Concepts_v3:")
    for concept in record.get("concepts_v3", []):
        print(f"  - {concept}")

    print("="*10)
    print("\n🔧 Aspects:")
    for aspect in record.get("aspects", []):
        print(f"  - {aspect}")

    print("\n🔧 Aspects_V2_:")
    for aspect in record.get("aspects_v2", []):
        print(f"  - {aspect}")

Record #1
----------------------------------------------------------------------------------------------------
📌 Text:

('This past year, many folks left their jobs to find better, more worthwhile '
 'opportunities. Director of Sales Max Johns shares 3 tips for anyone hoping '
 'to migrate into the fastest growing field; #machinelearning. #ai '
 '#datacentric #jobs #gethired https://t.co/ZEZxyPtboZ')
🌀 Sarcasm Detection: not_sarcastic

🔍 Named  (spaCy):
  - This past year (DATE)
  - Max Johns (PERSON)
  - 3 (CARDINAL)

🔍 Named Entities_V2_(HuggingFace):
  - Max Johns [PER] (score=1.00)

🧠 Concepts:
  - jobs_and_careers
  - retail_ecommerce
  - transportation_logistics

🧠 Concepts_V2_:

🔧 Aspects:
  - many folks
  - their jobs
  - better, more worthwhile opportunities
  - Sales
  - 3 tips
  - anyone
  - the fastest growing field
  - machinelearning

🔧 Aspects_V2_:
  - many folks
  - jobs
  - worthwhile opportunities
  - tips
  - growing field
Record #2
----------------------------------

## old data

In [ ]:
import pprint
  # Process and print results
for i, record in enumerate(data[:10]):
    print("=" * 100)
    print(f"Record #{i+1}")
    print("-" * 100)
    print(f"📌 Text:\n")
    pprint.pprint(f"{record['text']}")

    print("="*10)
    print(f"🌀 Sarcasm Detection: {record.get('sarcasm_label', 'N/A')}")

    print("="*10)
    print("\n🔍 Named Entities:")
    for ent in record.get("named_entities", []):
        print(f"  - {ent['entity']} ({ent['label']})")

    print("\n🔍 Named Entities_V2_:")
    for ent in record.get("named_entities_v2", []):
        print(f"  - {ent['entity']} ({ent['label']})")

    print("="*10)
    print("\n🧠 Concepts:")
    for concept in record.get("concepts", []):
        print(f"  - {concept}")

    print("\n🧠 Concepts_V2_:")
    for concept in record.get("concepts_v2", []):
        print(f"  - {concept}")

    print("="*10)
    print("\n🔧 Aspects:")
    for aspect in record.get("aspects", []):
        print(f"  - {aspect}")

    print("\n🔧 Aspects_V2_:")
    for aspect in record.get("aspects_v2", []):
        print(f"  - {aspect}")

Record #1
----------------------------------------------------------------------------------------------------
📌 Text:

('This past year, many folks left their jobs to find better, more worthwhile '
 'opportunities. Director of Sales Max Johns shares 3 tips for anyone hoping '
 'to migrate into the fastest growing field; #machinelearning. #ai '
 '#datacentric #jobs #gethired https://t.co/ZEZxyPtboZ')
🌀 Sarcasm Detection: not_sarcastic

🔍 Named Entities:
  - This past year (DATE)
  - Sales Max Johns (PERSON)
  - 3 (CARDINAL)
  - # (CARDINAL)
  - #ai # (MONEY)

🧠 Concepts:
  - ai_tech
  - jobs_and_careers

🔧 Aspects:
  - many folks
  - their jobs
  - better, more worthwhile opportunities
  - Director
  - Sales Max Johns
  - 3 tips
  - anyone
  - the fastest growing field
  - #machinelearning
  - datacentric
  - #jobs
  - #gethired https://t.co/ZEZxyPtboZ
Record #2
----------------------------------------------------------------------------------------------------
📌 Text:

('Will technolo